In [44]:
import pandas as pd 
# !pip install requests
import requests 
import numpy as np
import lxml
from pprint import pprint

In [45]:
url = "https://transfermarket.p.rapidapi.com/statistic/list-most-valuable-clubs"

headers = {
	"X-RapidAPI-Key": "1fe00a4fbdmsh972d29881773d57p1fda11jsn1ea3448c32ec",
	"X-RapidAPI-Host": "transfermarket.p.rapidapi.com"
}

querystring = {"domain":"de"}
response = requests.request("GET", url, headers=headers, params=querystring)
    
api_response = response.json()

In [46]:
#make a dataframe from the api_response
response_df = pd.DataFrame(api_response['teams'])

#drop unneeded columns 
response_df.drop(['number', 'countryImage','clubImage', 'marketValue', 'marketValueNumeral', 'progression'], axis=1, inplace=True)

#rename columns
response_df.rename(columns={'clubName':'Team', 'marketValueUnformatted':'Market_Value', 'countryName':'Country'}, inplace=True)

response_df.rename(columns={'competitionID': 'League_ID', 
                            'competitionName': 'League', 
                            'avgAge': 'Average_Age', 
                            'marketValueCurrency': 'Currency'}, inplace=True)

In [47]:
response_df

,id,Country,League_ID,League,Team,Average_Age,Currency,Market_Value
0,281,England,GB1,Premier League,Manchester City,27.20,€,1048750000
1,631,England,GB1,Premier League,FC Chelsea,25.50,€,1046500000
2,27,Deutschland,L1,Bundesliga,Bayern München,26.70,€,995700000
3,31,England,GB1,Premier League,Liverpool,27.00,€,931000000
4,583,Frankreich,FR1,Ligue 1,Paris SG,25.60,€,889050000
...,...,...,...,...,...,...,...,...
95,419,Kroatien,KR1,SuperSport HNL,Dinamo Zagreb,25.80,€,97300000
96,44,Deutschland,L1,Bundesliga,Hertha BSC,26.60,€,97200000
97,1132,England,GB2,Championship,FC Burnley,25.40,€,96550000
98,3,Deutschland,L1,Bundesliga,1.FC Köln,25.00,€,96025000


In [48]:
# Collect Lat/Long then merge to response_df
# Enter 'pip install geopy' to command to install geopy

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-app")
countries = response_df['Country'].unique()
lat=[]
long=[]

for c in countries:
    location = geolocator.geocode(c)
    if location:
        lat.append(location.latitude)
        long.append(location.longitude)
    else:
        lat.append(None)
        long.append(None)
countries_df = pd.DataFrame({"Country":countries,
                            "Lat":lat,
                            "Long":long})


In [49]:
countries_df

,Country,Lat,Long
0,England,52.531021,-1.264906
1,Deutschland,51.163818,10.447831
2,Frankreich,46.603354,1.888334
3,Spanien,39.326068,-4.837979
4,Italien,42.638426,12.674297
5,Monaco,43.732349,7.427683
6,Portugal,39.662165,-8.135352
7,Niederlande,52.243498,5.634323
8,Österreich,47.593970,14.124560
9,Türkei,38.959759,34.924965


In [50]:
#Merge main data with lat/long
response_df=pd.merge(response_df,countries_df,how='left',on='Country')
response_df.head()

,id,Country,League_ID,League,Team,Average_Age,Currency,Market_Value,Lat,Long
0,281,England,GB1,Premier League,Manchester City,27.20,€,1048750000,52.531021,-1.264906
1,631,England,GB1,Premier League,FC Chelsea,25.50,€,1046500000,52.531021,-1.264906
2,27,Deutschland,L1,Bundesliga,Bayern München,26.70,€,995700000,51.163818,10.447831
3,31,England,GB1,Premier League,Liverpool,27.00,€,931000000,52.531021,-1.264906
4,583,Frankreich,FR1,Ligue 1,Paris SG,25.60,€,889050000,46.603354,1.888334


In [51]:
# Collect performance data from fbef website
Ligue_1_League_ranking  = pd.read_html('https://fbref.com/en/comps/13/Ligue-1-Stats')[0]
Serie_A_rankings = pd.read_html('https://fbref.com/en/comps/11/Serie-A-Stats')[0]
Bundesliga_rankings = pd.read_html('https://fbref.com/en/comps/20/Bundesliga-Stats')[0]
Laliga_rankings = pd.read_html('https://fbref.com/en/comps/12/La-Liga-Stats')[0]
Premier_rankings = pd.read_html('https://fbref.com/en/comps/9/Premier-League-Stats')[0]
Liga_Portugal_rankings = pd.read_html('https://fbref.com/en/comps/32/Primeira-Liga-Stats')[0]
Eredivisie_rankings = pd.read_html('https://fbref.com/en/comps/23/Eredivisie-Stats')[0]
Super_Lig_rankings = pd.read_html('https://fbref.com/en/comps/26/Super-Lig-Stats')[0]
Jupiler_Pro_League_rankings = pd.read_html('https://fbref.com/en/comps/37/Belgian-First-Division-A-Stats')[0]
Campeonato_rankings = pd.read_html('https://fbref.com/en/comps/24/Serie-A-Stats')[0]
Scottish_Premiership_rankings = pd.read_html('https://fbref.com/en/comps/40/Scottish-Premiership-Stats')[0]
Championship_rankings = pd.read_html('https://fbref.com/en/comps/10/Championship-Stats')[0]
Super_League_1_rankings = pd.read_html('https://fbref.com/en/comps/27/Super-League-Greece-Stats')[0]
Campeonato_Mineiro_rankings=pd.read_html('https://fbref.com/en/comps/38/Serie-B-Stats')[0]
Liga_Professional_de_Futbol_rankings=pd.read_html('https://fbref.com/en/comps/21/Primera-Division-Stats')[0]
Supersport_HNL_rankings=pd.read_html('https://fbref.com/en/comps/63/1-HNL-Stats')[0]
Austria_rankings = pd.read_html('https://fbref.com/en/comps/56/Austrian-Bundesliga-Stats')[0]
Russia_rankings = pd.read_html('https://fbref.com/en/comps/30/Russian-Premier-League-Stats')[0]

In [52]:
# Union to 1 DataFrame
performance_df = pd.concat([Ligue_1_League_ranking,
                           Serie_A_rankings,
                           Bundesliga_rankings,
                           Laliga_rankings,
                           Premier_rankings,
                           Liga_Portugal_rankings,
                           Eredivisie_rankings,
                           Super_Lig_rankings,
                           Jupiler_Pro_League_rankings,
                           Campeonato_rankings,
                           Scottish_Premiership_rankings,
                           Championship_rankings,
                           Super_League_1_rankings,
                           Campeonato_Mineiro_rankings,
                           Liga_Professional_de_Futbol_rankings,
                           Supersport_HNL_rankings,
                           Austria_rankings,
                           Russia_rankings])

# Select columns
performance_df=performance_df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP']]
performance_df =performance_df.rename(columns = {'Pts/MP': 'Pts_per_Match'})

performance_df

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts_per_Match
0,1.0,Paris S-G,23.0,17.0,3.0,3.0,55.0,20.0,35.0,54.0,2.35
1,2.0,Marseille,23.0,15.0,4.0,4.0,45.0,20.0,25.0,49.0,2.13
2,3.0,Monaco,23.0,14.0,5.0,4.0,51.0,30.0,21.0,47.0,2.04
3,4.0,Lens,23.0,13.0,7.0,3.0,35.0,18.0,17.0,46.0,2.00
4,5.0,Lille,23.0,12.0,5.0,6.0,40.0,27.0,13.0,41.0,1.78
...,...,...,...,...,...,...,...,...,...,...,...
11,12.0,Nizhny Novgorod,17.0,5.0,4.0,8.0,22.0,29.0,-7.0,19.0,1.12
12,13.0,Fakel Voronezh,17.0,1.0,10.0,6.0,21.0,33.0,-12.0,13.0,0.76
13,14.0,Loko Moscow,17.0,3.0,4.0,10.0,20.0,34.0,-14.0,13.0,0.76
14,15.0,FC Khimki,17.0,3.0,3.0,11.0,16.0,36.0,-20.0,12.0,0.71


In [53]:
# Clean performance_df['Squad'] to match with response_df['Team'] before merging
response_df['Team']=[row.replace('1.FC Köln','Köln') for row in response_df['Team']]
response_df['Team']=[row.replace('1.FSV Mainz 05','Mainz 05') for row in response_df['Team']]
response_df['Team']=[row.replace('AC Florenz','Fiorentina') for row in response_df['Team']]
response_df['Team']=[row.replace('AC Mailand','Milan') for row in response_df['Team']]
response_df['Team']=[row.replace('AS Monaco','Monaco') for row in response_df['Team']]
response_df['Team']=[row.replace('AS Rom','Roma') for row in response_df['Team']]
response_df['Team']=[row.replace('Aston Villa','Aston Villa') for row in response_df['Team']]
response_df['Team']=[row.replace('Atal. Bergamo','Atalanta') for row in response_df['Team']]
response_df['Team']=[row.replace('Athletic Bilbao','Athletic Club') for row in response_df['Team']]
response_df['Team']=[row.replace('Atlético Madrid','Atlético Madrid') for row in response_df['Team']]
response_df['Team']=[row.replace('B. Leverkusen','Leverkusen') for row in response_df['Team']]
response_df['Team']=[row.replace('Bayern München','Bayern Munich') for row in response_df['Team']]
response_df['Team']=[row.replace('Betis Sevilla','Sevilla') for row in response_df['Team']]
response_df['Team']=[row.replace('Bor. Dortmund','Dortmund') for row in response_df['Team']]
response_df['Team']=[row.replace("Bor. M'gladbach","M'Gladbach") for row in response_df['Team']]
response_df['Team']=[row.replace('Bournemouth','Bournemouth') for row in response_df['Team']]
response_df['Team']=[row.replace('Brighton','Brighton') for row in response_df['Team']]
response_df['Team']=[row.replace('CA Osasuna','Osasuna') for row in response_df['Team']]
response_df['Team']=[row.replace('Celta Vigo','Celta Vigo') for row in response_df['Team']]
response_df['Team']=[row.replace('Crystal Palace','Crystal Palace') for row in response_df['Team']]
response_df['Team']=[row.replace('E. Frankfurt','Eint Frankfurt') for row in response_df['Team']]
response_df['Team']=[row.replace('Esp. Barcelona','Barcelona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Arsenal','Arsenal') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Augsburg','Augsburg') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Barcelona','Barcelona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Bologna','Bologna') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Brentford','Brentford') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Chelsea','Chelsea') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Everton','Everton') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Fulham','Fulham') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Getafe','Getafe') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Girona','Girona') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Nantes','Nantes') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Sevilla','Sevilla') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Turin','Torino') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Valencia','Valencia') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Villarreal','Villarreal') for row in response_df['Team']]
response_df['Team']=[row.replace('Hertha BSC','Hertha BSC') for row in response_df['Team']]
response_df['Team']=[row.replace('Inter Mailand','Inter') for row in response_df['Team']]
response_df['Team']=[row.replace('Juventus Turin','Juventus') for row in response_df['Team']]
response_df['Team']=[row.replace('Lazio Rom','Lazio') for row in response_df['Team']]
response_df['Team']=[row.replace('Leeds United','Leeds United') for row in response_df['Team']]
response_df['Team']=[row.replace('Leicester City','Leicester City') for row in response_df['Team']]
response_df['Team']=[row.replace('Liverpool','Liverpool') for row in response_df['Team']]
response_df['Team']=[row.replace('LOSC Lille','Lille') for row in response_df['Team']]
response_df['Team']=[row.replace('Manchester City','Manchester City') for row in response_df['Team']]
response_df['Team']=[row.replace('Manchester Utd.','Manchester Utd') for row in response_df['Team']]
response_df['Team']=[row.replace('Montpellier HSC','Montpellier') for row in response_df['Team']]
response_df['Team']=[row.replace('Monza','Monza') for row in response_df['Team']]
response_df['Team']=[row.replace('Newcastle Utd.','Newcastle Utd') for row in response_df['Team']]
response_df['Team']=[row.replace('Nottm Forest',"Nott'ham Forest") for row in response_df['Team']]
response_df['Team']=[row.replace('OGC Nizza','Nice') for row in response_df['Team']]
response_df['Team']=[row.replace('Ol. Marseille','Marseille') for row in response_df['Team']]
response_df['Team']=[row.replace('Olympique Lyon','Lyon') for row in response_df['Team']]
response_df['Team']=[row.replace('Paris SG','Paris S-G') for row in response_df['Team']]
response_df['Team']=[row.replace('RB Leipzig','RB Leipzig') for row in response_df['Team']]
response_df['Team']=[row.replace('RC Lens','Lens') for row in response_df['Team']]
response_df['Team']=[row.replace('Real Madrid','Real Madrid') for row in response_df['Team']]
response_df['Team']=[row.replace('Real Sociedad','Real Sociedad') for row in response_df['Team']]
response_df['Team']=[row.replace('Rennes','Rennes') for row in response_df['Team']]
response_df['Team']=[row.replace('SC Freiburg','Freiburg') for row in response_df['Team']]
response_df['Team']=[row.replace('Southampton','Southampton') for row in response_df['Team']]
response_df['Team']=[row.replace('SSC Neapel','Napoli') for row in response_df['Team']]
response_df['Team']=[row.replace('Tottenham','Tottenham') for row in response_df['Team']]
response_df['Team']=[row.replace('TSG Hoffenheim','Hoffenheim') for row in response_df['Team']]
response_df['Team']=[row.replace('UD Almería','Almería') for row in response_df['Team']]
response_df['Team']=[row.replace('Udinese Calcio','Udinese') for row in response_df['Team']]
response_df['Team']=[row.replace('Union Berlin','Union Berlin') for row in response_df['Team']]
response_df['Team']=[row.replace('US Salernitana','Salernitana') for row in response_df['Team']]
response_df['Team']=[row.replace('US Sassuolo','Sassuolo') for row in response_df['Team']]
response_df['Team']=[row.replace('VfB Stuttgart','Stuttgart') for row in response_df['Team']]
response_df['Team']=[row.replace('VfL Wolfsburg','Wolfsburg') for row in response_df['Team']]
response_df['Team']=[row.replace('West Ham Utd.','West Ham') for row in response_df['Team']]
response_df['Team']=[row.replace('Wolverhampton','Wolves') for row in response_df['Team']]
response_df['Team']=[row.replace('Benfica','Benfica') for row in response_df['Team']]
response_df['Team']=[row.replace('Ajax Amsterdam','Ajax') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Porto','Porto') for row in response_df['Team']]
response_df['Team']=[row.replace('Sporting','Sporting CP') for row in response_df['Team']]
response_df['Team']=[row.replace('RB Salzburg','RB Salzburg') for row in response_df['Team']]
response_df['Team']=[row.replace('PSV Eindhoven','PSV Eindhoven') for row in response_df['Team']]
response_df['Team']=[row.replace('Fenerbahce','Fenerbahçe') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Brügge','Club Brugge') for row in response_df['Team']]
response_df['Team']=[row.replace('Galatasaray','Galatasaray') for row in response_df['Team']]
response_df['Team']=[row.replace('Flamengo','Flamengo') for row in response_df['Team']]
response_df['Team']=[row.replace('Zenit S-Pb','Zenit') for row in response_df['Team']]
response_df['Team']=[row.replace('Feyenoord','Feyenoord') for row in response_df['Team']]
response_df['Team']=[row.replace('Trabzonspor','Trabzonspor') for row in response_df['Team']]
response_df['Team']=[row.replace('Glasgow Rangers','Rangers') for row in response_df['Team']]
response_df['Team']=[row.replace('Palmeiras','Palmeiras') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Watford','Watford') for row in response_df['Team']]
response_df['Team']=[row.replace('Besiktas','Beşiktaş') for row in response_df['Team']]
response_df['Team']=[row.replace('Atlético-MG','Atlético Mineiro') for row in response_df['Team']]
response_df['Team']=[row.replace('SC Braga','Braga') for row in response_df['Team']]
response_df['Team']=[row.replace('Celtic Glasgow','Celtic') for row in response_df['Team']]
response_df['Team']=[row.replace('Olymp. Piräus','Olympiacos') for row in response_df['Team']]
response_df['Team']=[row.replace('Norwich City','Norwich City') for row in response_df['Team']]
response_df['Team']=[row.replace('KRC Genk ','Genk') for row in response_df['Team']]
response_df['Team']=[row.replace('River Plate','River Plate') for row in response_df['Team']]
response_df['Team']=[row.replace('Dinamo Zagreb','Dinamo Zagreb') for row in response_df['Team']]
response_df['Team']=[row.replace('FC Burnley','Burnley') for row in response_df['Team']]

response_df.head()

,id,Country,League_ID,League,Team,Average_Age,Currency,Market_Value,Lat,Long
0,281,England,GB1,Premier League,Manchester City,27.20,€,1048750000,52.531021,-1.264906
1,631,England,GB1,Premier League,Chelsea,25.50,€,1046500000,52.531021,-1.264906
2,27,Deutschland,L1,Bundesliga,Bayern Munich,26.70,€,995700000,51.163818,10.447831
3,31,England,GB1,Premier League,Liverpool,27.00,€,931000000,52.531021,-1.264906
4,583,Frankreich,FR1,Ligue 1,Paris S-G,25.60,€,889050000,46.603354,1.888334


In [54]:
# Merge performance_df with response_df through Squad and Team
response_df=pd.merge(response_df,performance_df,how='left',left_on='Team',right_on='Squad')
response_df

,id,Country,League_ID,League,Team,Average_Age,Currency,Market_Value,Lat,Long,...,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts_per_Match
0,281,England,GB1,Premier League,Manchester City,27.20,€,1048750000,52.531021,-1.264906,...,Manchester City,24.0,16.0,4.0,4.0,60.0,24.0,36.0,52.0,2.17
1,631,England,GB1,Premier League,Chelsea,25.50,€,1046500000,52.531021,-1.264906,...,Chelsea,23.0,8.0,7.0,8.0,23.0,23.0,0.0,31.0,1.35
2,27,Deutschland,L1,Bundesliga,Bayern Munich,26.70,€,995700000,51.163818,10.447831,...,Bayern Munich,21.0,12.0,7.0,2.0,61.0,21.0,40.0,43.0,2.05
3,31,England,GB1,Premier League,Liverpool,27.00,€,931000000,52.531021,-1.264906,...,Liverpool,22.0,10.0,5.0,7.0,38.0,28.0,10.0,35.0,1.59
4,583,Frankreich,FR1,Ligue 1,Paris S-G,25.60,€,889050000,46.603354,1.888334,...,Paris S-G,23.0,17.0,3.0,3.0,55.0,20.0,35.0,54.0,2.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,419,Kroatien,KR1,SuperSport HNL,Dinamo Zagreb,25.80,€,97300000,45.365844,15.657521,...,Dinamo Zagreb,20.0,15.0,4.0,1.0,52.0,19.0,33.0,49.0,2.45
97,44,Deutschland,L1,Bundesliga,Hertha BSC,26.60,€,97200000,51.163818,10.447831,...,Hertha BSC,20.0,4.0,5.0,11.0,24.0,36.0,-12.0,17.0,0.85
98,1132,England,GB2,Championship,Burnley,25.40,€,96550000,52.531021,-1.264906,...,Burnley,32.0,21.0,9.0,2.0,63.0,27.0,36.0,72.0,2.25
99,3,Deutschland,L1,Bundesliga,Köln,25.00,€,96025000,51.163818,10.447831,...,Köln,21.0,6.0,8.0,7.0,32.0,34.0,-2.0,26.0,1.24


In [55]:
# Drop Squad column as contain same value as Team
response_df.drop(['Squad'], axis=1, inplace=True)

In [56]:
response_df

,id,Country,League_ID,League,Team,Average_Age,Currency,Market_Value,Lat,Long,Rk,MP,W,D,L,GF,GA,GD,Pts,Pts_per_Match
0,281,England,GB1,Premier League,Manchester City,27.20,€,1048750000,52.531021,-1.264906,2.0,24.0,16.0,4.0,4.0,60.0,24.0,36.0,52.0,2.17
1,631,England,GB1,Premier League,Chelsea,25.50,€,1046500000,52.531021,-1.264906,10.0,23.0,8.0,7.0,8.0,23.0,23.0,0.0,31.0,1.35
2,27,Deutschland,L1,Bundesliga,Bayern Munich,26.70,€,995700000,51.163818,10.447831,1.0,21.0,12.0,7.0,2.0,61.0,21.0,40.0,43.0,2.05
3,31,England,GB1,Premier League,Liverpool,27.00,€,931000000,52.531021,-1.264906,8.0,22.0,10.0,5.0,7.0,38.0,28.0,10.0,35.0,1.59
4,583,Frankreich,FR1,Ligue 1,Paris S-G,25.60,€,889050000,46.603354,1.888334,1.0,23.0,17.0,3.0,3.0,55.0,20.0,35.0,54.0,2.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,419,Kroatien,KR1,SuperSport HNL,Dinamo Zagreb,25.80,€,97300000,45.365844,15.657521,1.0,20.0,15.0,4.0,1.0,52.0,19.0,33.0,49.0,2.45
97,44,Deutschland,L1,Bundesliga,Hertha BSC,26.60,€,97200000,51.163818,10.447831,17.0,20.0,4.0,5.0,11.0,24.0,36.0,-12.0,17.0,0.85
98,1132,England,GB2,Championship,Burnley,25.40,€,96550000,52.531021,-1.264906,1.0,32.0,21.0,9.0,2.0,63.0,27.0,36.0,72.0,2.25
99,3,Deutschland,L1,Bundesliga,Köln,25.00,€,96025000,51.163818,10.447831,11.0,21.0,6.0,8.0,7.0,32.0,34.0,-2.0,26.0,1.24


In [66]:
# Change the type of columns

for col in ['Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']:
    response_df[col] = response_df[col].astype('int')

In [67]:
response_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             101 non-null    object 
 1   Country        101 non-null    object 
 2   League_ID      101 non-null    object 
 3   League         101 non-null    object 
 4   Team           101 non-null    object 
 5   Average_Age    101 non-null    object 
 6   Currency       101 non-null    object 
 7   Market_Value   101 non-null    int64  
 8   Lat            101 non-null    float64
 9   Long           101 non-null    float64
 10  Rk             101 non-null    int32  
 11  MP             101 non-null    int32  
 12  W              101 non-null    int32  
 13  D              101 non-null    int32  
 14  L              101 non-null    int32  
 15  GF             101 non-null    int32  
 16  GA             101 non-null    int32  
 17  GD             101 non-null    int32  
 18  Pts       

In [68]:
# Save reponse_df to a csv
response_df.to_csv('../Resources/cleaned_data.csv',index=False, encoding='utf-8-sig')